<a href="https://colab.research.google.com/github/merrymasti015/CAP2022GRP09/blob/main/PCAMZC321_Group09_FireEventDetection_from_VDO_InceptionResNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Apply model on vdo having no class


### Apply model on vdo having no class

In [ ]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pickle


import cv2
import math
import os, os.path
from pathlib import Path
from glob import glob
from scipy import stats as s

from moviepy.editor import *   ###VideoFileClip

from ipywidgets import Video # Display VDO 

 




Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2187264/45929032 bytes (4.8%)5038080/45929032 bytes (11.0%)8028160/45929032 bytes (17.5%)11034624/45929032 bytes (24.0%)13950976/45929032 bytes (30.4%)16990208/45929032 bytes (37.0%)20234240/45929032 bytes (44.1%)23142400/45929032 bytes (50.4%)26443776/45929032 bytes (57.6%)29163520/45929032 bytes (63.5%)31842304/45929032 bytes (69.3%)34398208/45929032 bytes (74.9%)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

ModelSaveFolder = "/content/gdrive/MyDrive/Fire_ImageDataSet/ModelSave"


Drive = "/content/gdrive/MyDrive/"
## Define root folder
RootFolder = Drive+"Fire_ImageDataSet/OutputFiles"

### Where to save models

ModelSaveFolder = Drive+"Fire_ImageDataSet/ModelSave"

print(ModelSaveFolder)

VDOFolder = Drive+"/Fire_ImageDataSet/VDO_fire"  # where is my VDO file ?




Mounted at /content/gdrive
/content/gdrive/MyDrive/Fire_ImageDataSet/ModelSave


In [ ]:
FramesVdo  = VDOFolder+"/VDOFrames1"
if not os.path.exists(FramesVdo):
   os.makedirs(FramesVdo) 
FramesVdo

'/content/gdrive/MyDrive//Fire_ImageDataSet/VDO_fire/VDOFrames1'

In [ ]:

if not os.listdir(FramesVdo):
    print ("empty")
else:
    print ("Not empty")

Not empty


In [ ]:
# Numeric to semantic labels 
model_class = ['Fire', 'No Fire']
model_class


['Fire', 'No Fire']

In [ ]:
model_class[0]

'Fire'

## Pre Processing 

In [ ]:

video_title = "VDO-1"

input_video_file_path = f'{VDOFolder}/{video_title}.mp4'
input_video_file_path


'/content/gdrive/MyDrive//Fire_ImageDataSet/VDO_fire/VDO-1.mp4'

In [ ]:
#Video.from_file(input_video_file_path)
Video.from_file(input_video_file_path , width = 320 , height = 320)

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\xe3\xaa_mdat\x00\x…

In [ ]:
from ipywidgets import Widget
Widget.close_all()

In [ ]:
def Pre_Processing(input_video_file_path,FramesVdo):
    
    flag =''
    image_height  = 150
    image_width   = 150
    
    EXTENSION = ".jpg"
    curr_frame = 0
    imageKount = 0
    KPS = 5 # Target Keyframes Per Second
    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(input_video_file_path)
    # Set frames-per-second for capture
    fps = round(video_reader.get(cv2.CAP_PROP_FPS))
    print('FPS = ' ,fps)
    hop = round(fps / KPS)
    #curr_frame = 5
    prediction_images = []
    
    ## Check if VDOFrames is empty or Not --- If empty then frames will be stored else NOT
    ## OR we can delete all files in VDOFrames folder unconditionally 
    
    if not os.listdir(FramesVdo):
        print ("empty VDOFrames folder")
        flag = "empty"
    else:
        print ("Not empty")
    
    
    
    
    while True: 

        # Reading The Frame
        status, frame = video_reader.read() 
        
        if not status:
            break

        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        
        ## if VDOFrames is empty i.e. flag = "empty" then write
        
        if flag == 'empty':
           
           temp = Path(input_video_file_path).stem # Find file name without extension
           filename ="/"+ temp+"_frame%d.jpg" % curr_frame
           Outimage = os.path.join(FramesVdo+filename)
           cv2.imwrite(Outimage, resized_frame)   # save frame as JPEG file
            
            
            
            
        ## PreProcessing  
        normalized_frame = resized_frame / 255
        # appending the image to the image list
        prediction_images.append(normalized_frame)
        imageKount +=1
        curr_frame +=1
      
    
    # converting the list to numpy array
    prediction_images = np.array(prediction_images)
        
    # shape of the array
    print(prediction_images.shape)
    print(curr_frame)
    print(imageKount)   
    #return batch_data
    return prediction_images  # Return prediction_images numpy array


In [ ]:
%%time
prediction_images = Pre_Processing(input_video_file_path,FramesVdo)

FPS =  25
Not empty
(1872, 150, 150, 3)
1872
1872
CPU times: user 15.2 s, sys: 1.06 s, total: 16.3 s
Wall time: 6.98 s


## End of Pre Processing 

## Event Prediction 

### Load  model from disk

In [ ]:
from tensorflow import keras

In [ ]:
#model_name = 'InceptionResNetV2_Prediction_fine_tuned.h5'
model_name = 'InceptionResNetV2_suns.h5'
modeIV2 = keras.models.load_model(ModelSaveFolder+'/'+model_name)




In [ ]:
predict = []        
# predict_classes() function model in order to predict the class values for each instance in the array.
#prediction = np.round(modeIV2.predict_classes(prediction_images)).flatten()
prediction = np.argmax(modeIV2.predict(prediction_images), axis=-1)
#np.round(model.predict(img)).flatten()

# prediction is an arrary containing frame by frame class prediction - here either Basketball', 'SoccerPenalty'

print(prediction)

[0 0 0 ... 0 0 0]


#### Pickle is the standard way of serializing objects in Python.

#### Use the pickle operation to serialize  machine learning algorithms and save the serialized format to a file.

#### Later we can load this file to deserialize your model and use it to make new predictions.

#### Serialization is the process of converting an object into a stream of bytes to store the object or transmit it to memory, or a file.

### Save prediction array using pickle dump

In [ ]:
import pickle

with open('IV2Prediction_j', 'wb') as fp:
    pickle.dump(prediction, fp)

In [ ]:
#To read it back:
with open ('IV2Prediction_j', 'rb') as fp:
    prediction = pickle.load(fp)

In [ ]:
print(prediction[5])

0


### Function To Predict on Live Videos 

In [ ]:

print(input_video_file_path)
print(FramesVdo)




/content/gdrive/MyDrive//Fire_ImageDataSet/VDO_fire/VDO-1.mp4
/content/gdrive/MyDrive//Fire_ImageDataSet/VDO_fire/VDOFrames1


In [ ]:
prediction[17]

0

In [ ]:
x = len(prediction)
print(x)
# Accessing The Class Name using predicted label.
predicted_class_name = model_class[prediction[2]]
predicted_class_name

1872


'Fire'

In [ ]:
input_video_file_path

'/content/gdrive/MyDrive//Fire_ImageDataSet/VDO_fire/VDO-1.mp4'

#### cv2.putText() method is used to draw a text string on any image.

Syntax: cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])

Parameters:
image:           It is the image on which text is to be drawn.
text:            Text string to be drawn.
org:             It is the coordinates of the bottom-left corner of the text string in the image. The coordinates are represented as tuples of two values i.e. (X coordinate value, Y coordinate value).
font:            It denotes the font type. Some of font types are FONT_HERSHEY_SIMPLEX, FONT_HERSHEY_PLAIN, , etc.
fontScale:       Font scale factor that is multiplied by the font-specific base size.
color:           It is the color of text string to be drawn. For BGR, we pass a tuple. eg: (255, 0, 0) for blue color.
thickness:       It is the thickness of the line in px.
lineType:        This is an optional parameter.It gives the type of the line to be used.
bottomLeftOrigin: This is an optional parameter. When it is true, the image data origin is at the bottom-left corner. Otherwise, it is at the top-left corner.

Return Value: It returns an image.



In [ ]:
def predict_on_live_video(video_file_path, output_file_path,Algo):

    # Initialize a Deque Object with a fixed size which will be used to implement moving/rolling average functionality.
    #predicted_labels_probabilities_deque = deque(maxlen = window_size)
    
    frame_no = -1

    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(video_file_path)

    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(original_video_width)
    print(original_video_height)
    # Writing the Overlayed Video Files Using the VideoWriter Object
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))

    while True: 

        # Reading The Frame
        status, frame = video_reader.read() 

        if not status:
            break

        frame_no = frame_no+1
        
        
        # Using cv2.putText()
        
        # Accessing The Class Name using prediction list.
        predicted_class_name = model_class[prediction[frame_no]]
        #print(predicted_class_name)
        
        # Overlaying Class Name Text Ontop of the Frame
        cv2.putText(frame, predicted_class_name+'_(' + Algo + ')', (00, 200), cv2.FONT_HERSHEY_DUPLEX, 3.0, (0, 0, 255), 3)
        # Writing The Frame
        video_writer.write(frame)
    
    # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them. 
    video_reader.release()
    video_writer.release()

In [ ]:
ModelName = 'IV2'
output_video_file_path  = VDOFolder+'/'+video_title+'_'+ ModelName+'_Gr_9_Output.mp4'
print(output_video_file_path)
# Calling the predict_on_live_video method to start the Prediction.
predict_on_live_video(input_video_file_path, output_video_file_path , ModelName)


/content/gdrive/MyDrive//Fire_ImageDataSet/VDO_fire/VDO-1_IV2_Gr_9_Output.mp4
1280
720


### End